# Data Science Final Project - Shelter Animal Adoptions

## Questions:
## 1.  Are animal type, sex, age, and color significant predictors of shelter dog and cat adoption?
## 2.  What time of year are shelter dog and cat adoptions most popular?

## Import Packages

In [1]:
import pandas as pd   # for reading data, data wrangling, visualization, and analysis
import numpy as np   # for mathematical operations on arrays (in modeling)
from scipy import stats   # for independent chi-square (statistical tests)
import statsmodels as sm   # for mcnemar chi-square (statistical tests and models)
from statsmodels.stats.contingency_tables import mcnemar   # for mcnemar chi-square
import os   # to create "data" folder and convert df to csv

## Import Data

In [2]:
adoptions = pd.read_csv("Austin_Animal_Center_Outcomes.csv")

In [3]:
adoptions.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,5/8/2019 18:20,19-May,5/2/2017,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,7/18/2018 16:02,18-Jul,7/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,8/16/2020 11:38,20-Aug,8/16/2019,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2/13/2016 17:59,16-Feb,10/8/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,3/18/2014 11:47,14-Mar,3/12/2014,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


## Data Wrangling

### Keep only relevant columns

In [4]:
adoptions1 = adoptions[["MonthYear", "Outcome Type", "Animal Type", "Sex upon Outcome", "Age upon Outcome", "Color"]]

In [5]:
adoptions1.head()

,MonthYear,Outcome Type,Animal Type,Sex upon Outcome,Age upon Outcome,Color
0,19-May,Rto-Adopt,Cat,Neutered Male,2 years,Brown Tabby/White
1,18-Jul,Adoption,Dog,Neutered Male,1 year,White/Brown
2,20-Aug,Euthanasia,Other,Unknown,1 year,Gray
3,16-Feb,Adoption,Dog,Neutered Male,4 months,Buff
4,14-Mar,Transfer,Cat,Intact Male,6 days,Orange Tabby


### Split MonthYear into 2 columns

In [6]:
adoptions2 = adoptions1["MonthYear"].str.split("-", expand=True)

In [7]:
adoptions2.head

<bound method NDFrame.head of          0    1
0       19  May
1       18  Jul
2       20  Aug
3       16  Feb
4       14  Mar
...     ..  ...
136751  22  Feb
136752  22  Feb
136753  22  Feb
136754  22  Feb
136755  22  Feb

[136756 rows x 2 columns]>

### Add new month and year columns back to dataframe

In [8]:
adoptions3 = pd.concat([adoptions1, adoptions2], axis=1)

In [9]:
adoptions3.head()

,MonthYear,Outcome Type,Animal Type,Sex upon Outcome,Age upon Outcome,Color,0,1
0,19-May,Rto-Adopt,Cat,Neutered Male,2 years,Brown Tabby/White,19,May
1,18-Jul,Adoption,Dog,Neutered Male,1 year,White/Brown,18,Jul
2,20-Aug,Euthanasia,Other,Unknown,1 year,Gray,20,Aug
3,16-Feb,Adoption,Dog,Neutered Male,4 months,Buff,16,Feb
4,14-Mar,Transfer,Cat,Intact Male,6 days,Orange Tabby,14,Mar


### Rename new columns and drop original date column

In [10]:
adoptions3.rename(columns={0: "Year", 1: "Month"}, inplace=True)
adoptions3.drop(["MonthYear"], axis=1, inplace=True)

In [11]:
adoptions3.head()

,Outcome Type,Animal Type,Sex upon Outcome,Age upon Outcome,Color,Year,Month
0,Rto-Adopt,Cat,Neutered Male,2 years,Brown Tabby/White,19,May
1,Adoption,Dog,Neutered Male,1 year,White/Brown,18,Jul
2,Euthanasia,Other,Unknown,1 year,Gray,20,Aug
3,Adoption,Dog,Neutered Male,4 months,Buff,16,Feb
4,Transfer,Cat,Intact Male,6 days,Orange Tabby,14,Mar


### Group and Recode Sex upon Outcome column

In [12]:
adoptions3["Sex upon Outcome"].value_counts()

Neutered Male    48172
Spayed Female    43461
Intact Male      17371
Intact Female    16872
Unknown          10879
Name: Sex upon Outcome, dtype: int64

In [13]:
def sex (series): 
    if series == "Neutered Male" : 
        return "Male"
    if series == "Spayed Female": 
        return "Female"
    if series == "Intact Male" : 
        return "Male"
    if series == "Intact Female" : 
        return "Female"
    if series =="Unknown":
        return "Unknown"
    
adoptions3["Sex"] = adoptions3["Sex upon Outcome"].apply(sex)

In [14]:
adoptions3.head(10)

,Outcome Type,Animal Type,Sex upon Outcome,Age upon Outcome,Color,Year,Month,Sex
0,Rto-Adopt,Cat,Neutered Male,2 years,Brown Tabby/White,19,May,Male
1,Adoption,Dog,Neutered Male,1 year,White/Brown,18,Jul,Male
2,Euthanasia,Other,Unknown,1 year,Gray,20,Aug,Unknown
3,Adoption,Dog,Neutered Male,4 months,Buff,16,Feb,Male
4,Transfer,Cat,Intact Male,6 days,Orange Tabby,14,Mar,Male
5,Adoption,Dog,Spayed Female,7 years,Brown,20,Oct,Female
6,Adoption,Dog,Neutered Male,2 years,White/Brown,20,May,Male
7,Adoption,Cat,Neutered Male,2 months,Black,14,Oct,Male
8,Adoption,Cat,Neutered Male,2 months,White/Orange Tabby,14,Aug,Male
9,Adoption,Dog,Spayed Female,2 years,Black/White,21,Aug,Female


### Drop Sex upon Outcome column

In [15]:
adoptions3.drop(["Sex upon Outcome"], axis=1, inplace=True)

In [16]:
adoptions3.head()

,Outcome Type,Animal Type,Age upon Outcome,Color,Year,Month,Sex
0,Rto-Adopt,Cat,2 years,Brown Tabby/White,19,May,Male
1,Adoption,Dog,1 year,White/Brown,18,Jul,Male
2,Euthanasia,Other,1 year,Gray,20,Aug,Unknown
3,Adoption,Dog,4 months,Buff,16,Feb,Male
4,Transfer,Cat,6 days,Orange Tabby,14,Mar,Male


### Filter Animal Type to only include Dog and Cat

In [17]:
adoptions4 = adoptions3[(adoptions3["Animal Type"].isin(["Dog", "Cat"]))]

In [18]:
adoptions4.head(10)

,Outcome Type,Animal Type,Age upon Outcome,Color,Year,Month,Sex
0,Rto-Adopt,Cat,2 years,Brown Tabby/White,19,May,Male
1,Adoption,Dog,1 year,White/Brown,18,Jul,Male
3,Adoption,Dog,4 months,Buff,16,Feb,Male
4,Transfer,Cat,6 days,Orange Tabby,14,Mar,Male
5,Adoption,Dog,7 years,Brown,20,Oct,Female
6,Adoption,Dog,2 years,White/Brown,20,May,Male
7,Adoption,Cat,2 months,Black,14,Oct,Male
8,Adoption,Cat,2 months,White/Orange Tabby,14,Aug,Male
9,Adoption,Dog,2 years,Black/White,21,Aug,Female
10,Transfer,Cat,2 years,Black,14,Jul,Female


### Filter out partial years from Year column

In [19]:
adoptions4["Year"].value_counts().sort_index(ascending=True)

13     4265
14    17556
15    17606
16    16572
17    16518
18    15791
19    18861
20     9046
21    11267
22     1377
Name: Year, dtype: int64

In [20]:
adoptions5 = adoptions4[(adoptions4["Year"].isin(["14", "15", "16", "17", "18", "19", "20", "21"]))]

In [21]:
adoptions5["Year"].unique()

array(['19', '18', '16', '14', '20', '21', '17', '15'], dtype=object)

In [22]:
adoptions5["Year"].value_counts().sort_index(ascending=True)

14    17556
15    17606
16    16572
17    16518
18    15791
19    18861
20     9046
21    11267
Name: Year, dtype: int64

In [23]:
adoptions5.head()

,Outcome Type,Animal Type,Age upon Outcome,Color,Year,Month,Sex
0,Rto-Adopt,Cat,2 years,Brown Tabby/White,19,May,Male
1,Adoption,Dog,1 year,White/Brown,18,Jul,Male
3,Adoption,Dog,4 months,Buff,16,Feb,Male
4,Transfer,Cat,6 days,Orange Tabby,14,Mar,Male
5,Adoption,Dog,7 years,Brown,20,Oct,Female


### Drop outlier rows from Age upon Outcome column

In [24]:
adoptions5["Age upon Outcome"].value_counts()

1 year       19328
2 years      18392
2 months     16181
3 years       7430
3 months      6251
1 month       5639
4 years       4436
4 months      4170
5 years       4166
5 months      2994
6 years       2770
6 months      2674
8 years       2430
7 years       2396
3 weeks       2269
2 weeks       2185
8 months      1992
10 years      1980
10 months     1843
7 months      1634
4 weeks       1535
9 years       1352
9 months      1303
12 years       947
1 weeks        935
11 months      809
11 years       774
1 week         701
13 years       624
14 years       419
3 days         384
2 days         354
15 years       345
1 day          285
6 days         249
4 days         228
0 years        167
16 years       156
5 weeks        147
5 days         143
17 years        88
18 years        51
19 years        26
20 years        19
22 years         6
-1 years         6
24 years         1
23 years         1
-3 years         1
-2 years         1
Name: Age upon Outcome, dtype: int64

#### Begin by dropping the "22 years" row.

In [25]:
indexNames = adoptions5[adoptions5["Age upon Outcome"] == "22 years"].index

In [26]:
adoptions5.drop(indexNames, inplace=True)

C:\Users\bueno\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [27]:
adoptions5["Age upon Outcome"].value_counts()

1 year       19328
2 years      18392
2 months     16181
3 years       7430
3 months      6251
1 month       5639
4 years       4436
4 months      4170
5 years       4166
5 months      2994
6 years       2770
6 months      2674
8 years       2430
7 years       2396
3 weeks       2269
2 weeks       2185
8 months      1992
10 years      1980
10 months     1843
7 months      1634
4 weeks       1535
9 years       1352
9 months      1303
12 years       947
1 weeks        935
11 months      809
11 years       774
1 week         701
13 years       624
14 years       419
3 days         384
2 days         354
15 years       345
1 day          285
6 days         249
4 days         228
0 years        167
16 years       156
5 weeks        147
5 days         143
17 years        88
18 years        51
19 years        26
20 years        19
-1 years         6
24 years         1
23 years         1
-3 years         1
-2 years         1
Name: Age upon Outcome, dtype: int64

#### It worked!  Now drop the remaining 5 outlier rows.

In [28]:
indexNames1 = adoptions5[adoptions5["Age upon Outcome"] == "-1 years"].index
indexNames2 = adoptions5[adoptions5["Age upon Outcome"] == "23 years"].index
indexNames3 = adoptions5[adoptions5["Age upon Outcome"] == "-3 years"].index
indexNames4 = adoptions5[adoptions5["Age upon Outcome"] == "24 years"].index
indexNames5 = adoptions5[adoptions5["Age upon Outcome"] == "-2 years"].index

In [29]:
adoptions5.drop(indexNames1, inplace=True)
adoptions5.drop(indexNames2, inplace=True)
adoptions5.drop(indexNames3, inplace=True)
adoptions5.drop(indexNames4, inplace=True)
adoptions5.drop(indexNames5, inplace=True)

In [30]:
adoptions5["Age upon Outcome"].value_counts()

1 year       19328
2 years      18392
2 months     16181
3 years       7430
3 months      6251
1 month       5639
4 years       4436
4 months      4170
5 years       4166
5 months      2994
6 years       2770
6 months      2674
8 years       2430
7 years       2396
3 weeks       2269
2 weeks       2185
8 months      1992
10 years      1980
10 months     1843
7 months      1634
4 weeks       1535
9 years       1352
9 months      1303
12 years       947
1 weeks        935
11 months      809
11 years       774
1 week         701
13 years       624
14 years       419
3 days         384
2 days         354
15 years       345
1 day          285
6 days         249
4 days         228
0 years        167
16 years       156
5 weeks        147
5 days         143
17 years        88
18 years        51
19 years        26
20 years        19
Name: Age upon Outcome, dtype: int64

### Group and Recode Age upon Outcome column

In [31]:
def age (series): 
    if series == "1 year" : 
        return "0-1 years"
    if series == "2 years": 
        return "2-5 years"
    if series == "2 months" : 
        return "0-1 years"
    if series == "3 years" : 
        return "2-5 years"
    if series =="3 months":
        return "0-1 years"
    if series =="1 month":
        return "0-1 years"
    if series =="4 years":
        return "2-5 years"
    if series =="4 months":
        return "0-1 years"
    if series =="5 years":
        return "2-5 years"
    if series =="5 months":
        return "0-1 years"
    if series =="6 years":
        return "6-9 years"
    if series =="6 months":
        return "0-1 years"
    if series =="8 years":
        return "6-9 years"
    if series =="7 years":
        return "6-9 years"
    if series =="3 weeks":
        return "0-1 years"
    if series =="2 weeks":
        return "0-1 years"
    if series =="8 months":
        return "0-1 years"
    if series =="10 years":
        return "10-15 years"
    if series =="10 months":
        return "0-1 years"
    if series =="7 months":
        return "0-1 years"
    if series =="4 weeks":
        return "0-1 years"
    if series =="9 years":
        return "6-9 years"
    if series =="9 months":
        return "0-1 years"
    if series =="12 years":
        return "10-15 years"
    if series =="1 weeks":
        return "0-1 years"
    if series =="11 months":
        return "0-1 years"
    if series =="11 years":
        return "10-15 years"
    if series =="13 years":
        return "10-15 years"
    if series =="14 years":
        return "10-15 years"
    if series =="3 days":
        return "0-1 years"
    if series =="2 days":
        return "0-1 years"
    if series =="15 years":
        return "10-15 years"
    if series =="1 day":
        return "0-1 years"
    if series =="6 days":
        return "0-1 years"
    if series =="4 days":
        return "0-1 years"
    if series =="16 years":
        return "16+ years"
    if series =="5 weeks":
        return "0-1 years"
    if series =="5 days":
        return "0-1 years"
    if series =="17 years":
        return "16+ years"
    if series =="18 years":
        return "16+ years"
    if series =="19 years":
        return "16+ years"
    if series =="20 years":
        return "16+ years"
     
adoptions5["Age"] = adoptions5["Age upon Outcome"].apply(age)

<ipython-input-31-f9909a0aeb6b>:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adoptions5["Age"] = adoptions5["Age upon Outcome"].apply(age)


In [32]:
adoptions5["Age"].value_counts()

0-1 years      73532
2-5 years      34424
6-9 years       8948
10-15 years     5089
16+ years        340
Name: Age, dtype: int64

### Drop Age upon Outcome column

In [33]:
adoptions5.drop(["Age upon Outcome"], axis=1, inplace=True)

In [34]:
adoptions5.head()

,Outcome Type,Animal Type,Color,Year,Month,Sex,Age
0,Rto-Adopt,Cat,Brown Tabby/White,19,May,Male,2-5 years
1,Adoption,Dog,White/Brown,18,Jul,Male,0-1 years
3,Adoption,Dog,Buff,16,Feb,Male,0-1 years
4,Transfer,Cat,Orange Tabby,14,Mar,Male,0-1 years
5,Adoption,Dog,Brown,20,Oct,Female,6-9 years


### Recode "Outcome Type" column to Adoption or Not Adoption

In [35]:
adoptions5["Outcome Type"].value_counts()

Adoption           58910
Transfer           36797
Return to Owner    21731
Euthanasia          3687
Died                1033
Rto-Adopt            819
Disposal             137
Missing               65
Relocate               4
Name: Outcome Type, dtype: int64

In [36]:
adoptions5["Outcome Type"].value_counts().sum()

123183

In [37]:
def adopt (series): 
    if series == "Adoption" : 
        return 1
    else:
        return 0
    
adoptions5["AdoptionYN"] = adoptions5["Outcome Type"].apply(adopt)

<ipython-input-37-37f47585a9c5>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adoptions5["AdoptionYN"] = adoptions5["Outcome Type"].apply(adopt)


In [38]:
adoptions5.head()

,Outcome Type,Animal Type,Color,Year,Month,Sex,Age,AdoptionYN
0,Rto-Adopt,Cat,Brown Tabby/White,19,May,Male,2-5 years,0
1,Adoption,Dog,White/Brown,18,Jul,Male,0-1 years,1
3,Adoption,Dog,Buff,16,Feb,Male,0-1 years,1
4,Transfer,Cat,Orange Tabby,14,Mar,Male,0-1 years,0
5,Adoption,Dog,Brown,20,Oct,Female,6-9 years,1


In [39]:
adoptions5["AdoptionYN"].value_counts()

0    64291
1    58910
Name: AdoptionYN, dtype: int64

### Drop Outcome Type column

In [40]:
adoptions5.drop(["Outcome Type"], axis=1, inplace=True)

C:\Users\bueno\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [41]:
adoptions5.head()

,Animal Type,Color,Year,Month,Sex,Age,AdoptionYN
0,Cat,Brown Tabby/White,19,May,Male,2-5 years,0
1,Dog,White/Brown,18,Jul,Male,0-1 years,1
3,Dog,Buff,16,Feb,Male,0-1 years,1
4,Cat,Orange Tabby,14,Mar,Male,0-1 years,0
5,Dog,Brown,20,Oct,Female,6-9 years,1


### Recode Animal Type to a numeric variable

In [42]:
adoptions5.dtypes

Animal Type    object
Color          object
Year           object
Month          object
Sex            object
Age            object
AdoptionYN      int64
dtype: object

In [43]:
def type_recode (series):
    if series == "Cat":
        return 0
    if series == "Dog":
        return 1
    
adoptions5['typeR'] = adoptions5['Animal Type'].apply(type_recode)

<ipython-input-43-e6b217218a0c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adoptions5['typeR'] = adoptions5['Animal Type'].apply(type_recode)


In [44]:
adoptions5.head()

,Animal Type,Color,Year,Month,Sex,Age,AdoptionYN,typeR
0,Cat,Brown Tabby/White,19,May,Male,2-5 years,0,0
1,Dog,White/Brown,18,Jul,Male,0-1 years,1,1
3,Dog,Buff,16,Feb,Male,0-1 years,1,1
4,Cat,Orange Tabby,14,Mar,Male,0-1 years,0,0
5,Dog,Brown,20,Oct,Female,6-9 years,1,1


### Convert data type of Year to integer

In [45]:
adoptions5['Year'] = adoptions5['Year'].astype(int)

<ipython-input-45-f07139841f8e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adoptions5['Year'] = adoptions5['Year'].astype(int)


In [46]:
adoptions5["Year"].dtypes

dtype('int32')

### Drop "unknown" rows from Sex column

In [47]:
adoptions5["Sex"].value_counts()

Male       61827
Female     56912
Unknown     4462
Name: Sex, dtype: int64

In [48]:
indexNames = adoptions5[adoptions5["Sex"] == "Unknown"].index

In [49]:
adoptions5.drop(indexNames, inplace=True)

C:\Users\bueno\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [50]:
adoptions5["Sex"].value_counts()

Male      61827
Female    56912
Name: Sex, dtype: int64

### Recode Sex to a numeric variable

In [51]:
def sex_recode (series):
    if series == "Male":
        return 0
    if series == "Female":
        return 1
    
adoptions5['sexR'] = adoptions5['Sex'].apply(sex_recode)

<ipython-input-51-a5062100dc53>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adoptions5['sexR'] = adoptions5['Sex'].apply(sex_recode)


In [52]:
adoptions5.head()

,Animal Type,Color,Year,Month,Sex,Age,AdoptionYN,typeR,sexR
0,Cat,Brown Tabby/White,19,May,Male,2-5 years,0,0,0
1,Dog,White/Brown,18,Jul,Male,0-1 years,1,1,0
3,Dog,Buff,16,Feb,Male,0-1 years,1,1,0
4,Cat,Orange Tabby,14,Mar,Male,0-1 years,0,0,0
5,Dog,Brown,20,Oct,Female,6-9 years,1,1,1


### Recode Age to a numeric variable

In [53]:
adoptions5['Age'].value_counts()

0-1 years      70101
2-5 years      33931
6-9 years       8910
10-15 years     5054
16+ years        337
Name: Age, dtype: int64

In [54]:
adoptions5['Age'].dtypes

dtype('O')

In [55]:
def age_recode (series):
    if series == "0-1 years":
        return 0
    if series == "2-5 years":
        return 1
    if series == "6-9 years":
        return 2
    if series == "10-15 years":
        return 3
    if series == "16+ years":
        return 4
    
adoptions5['ageR'] = adoptions5['Age'].apply(age_recode)

<ipython-input-55-7bd947281773>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adoptions5['ageR'] = adoptions5['Age'].apply(age_recode)


In [56]:
adoptions5.head()

,Animal Type,Color,Year,Month,Sex,Age,AdoptionYN,typeR,sexR,ageR
0,Cat,Brown Tabby/White,19,May,Male,2-5 years,0,0,0,1.0
1,Dog,White/Brown,18,Jul,Male,0-1 years,1,1,0,0.0
3,Dog,Buff,16,Feb,Male,0-1 years,1,1,0,0.0
4,Cat,Orange Tabby,14,Mar,Male,0-1 years,0,0,0,0.0
5,Dog,Brown,20,Oct,Female,6-9 years,1,1,1,2.0


In [57]:
adoptions5['ageR'].value_counts()

0.0    70101
1.0    33931
2.0     8910
3.0     5054
4.0      337
Name: ageR, dtype: int64

In [58]:
adoptions5.dtypes

Animal Type     object
Color           object
Year             int32
Month           object
Sex             object
Age             object
AdoptionYN       int64
typeR            int64
sexR             int64
ageR           float64
dtype: object

### Group and Recode Color column

In [59]:
pd.set_option('display.max_rows', 600)
adoptions5['Color'].value_counts()

Black/White                    12771
Black                           9700
Brown Tabby                     6875
White                           4243
Brown/White                     4179
Tan/White                       3883
Brown Tabby/White               3629
Blue/White                      3457
White/Black                     3375
Orange Tabby                    3335
Tan                             3210
Brown                           2950
Tricolor                        2775
Black/Tan                       2772
Black/Brown                     2628
White/Brown                     2421
Tortie                          2225
Brown Brindle/White             2210
Blue                            2187
Calico                          2122
Brown/Black                     2004
White/Tan                       1900
Blue Tabby                      1815
Orange Tabby/White              1705
Torbie                          1446
Red/White                       1340
Red                             1338
T

#### Group by primary Color of animal

In [60]:
adoptions5.replace(['Agouti', 'Agouti/Brown Tabby', 'Agouti/Cream', 'Agouti/White'], 'Agouti', inplace =True)
adoptions5.replace(['Apricot', 'Apricot/Brown', 'Apricot/Tricolor', 'Apricot/White'], 'Apricot', inplace =True)
adoptions5.replace(['Black', 'Black Brindle', 'Black Brindle/Black', 'Black Brindle/Blue', 'Black Brindle/Blue Tick', 'Black Brindle/Brown', 'Black Brindle/Brown Brindle', 'Black Brindle/Tan', 'Black Brindle/White', 'Black Smoke', 'Black Smoke/Black', 'Black Smoke/Black Tabby', 'Black Smoke/Black Tiger', 'Black Smoke/Blue Tick', 'Black Smoke/Brown', 'Black Smoke/Brown Tabby', 'Black Smoke/Gray', 'Black Smoke/White', 'Black Tabby', 'Black Tabby/Black', 'Black Tabby/Gray', 'Black Tabby/Gray Tabby', 'Black Tabby/Orange', 'Black Tabby/White', 'Black Tiger', 'Black Tiger/White', 'Black/Black', 'Black/Black Brindle', 'Black/Black Smoke', 'Black/Black Tabby', 'Black/Blue', 'Black/Blue Merle', 'Black/Blue Tick', 'Black/Brown', 'Black/Brown Brindle', 'Black/Brown Merle', 'Black/Brown Tabby', 'Black/Buff', 'Black/Chocolate', 'Black/Cream', 'Black/Fawn', 'Black/Gold', 'Black/Gray', 'Black/Gray Tabby', 'Black/Orange', 'Black/Red', 'Black/Seal Point', 'Black/Silver', 'Black/Silver Tabby', 'Black/Tan', 'Black/Tortie', 'Black/Tricolor', 'Black/White','Black/Yellow', 'Black/Yellow Brindle'], 'Black', inplace =True)
adoptions5.replace(['Blue', 'Blue Cream', 'Blue Cream/Blue Tabby', 'Blue Cream/Blue Tiger', 'Blue Cream/Buff', 'Blue Cream/Tortie', 'Blue Cream/White', 'Blue Merle', 'Blue Merle/Black', 'Blue Merle/Blue Merle', 'Blue Merle/Brown', 'Blue Merle/Brown Brindle', 'Blue Merle/Cream', 'Blue Merle/Gray', 'Blue Merle/Red', 'Blue Merle/Red Merle', 'Blue Merle/Tan', 'Blue Merle/Tricolor', 'Blue Merle/White', 'Blue Point', 'Blue Point/Cream', 'Blue Point/Lynx Point', 'Blue Point/White', 'Blue Smoke', 'Blue Smoke/Brown', 'Blue Smoke/Gray', 'Blue Smoke/White', 'Blue Tabby', 'Blue Tabby/Black', 'Blue Tabby/Blue Cream', 'Blue Tabby/Brown', 'Blue Tabby/Buff', 'Blue Tabby/Cream', 'Blue Tabby/Orange', 'Blue Tabby/Tan', 'Blue Tabby/Tortie', 'Blue Tabby/White', 'Blue Tick', 'Blue Tick/Black', 'Blue Tick/Brown', 'Blue Tick/Brown Brindle', 'Blue Tick/Red', 'Blue Tick/Red Tick', 'Blue Tick/Tan', 'Blue Tick/Tricolor', 'Blue Tick/White', 'Blue Tiger', 'Blue Tiger/Tan', 'Blue Tiger/White', 'Blue/Black', 'Blue/Black Brindle', 'Blue/Blue', 'Blue/Blue Merle', 'Blue/Brown', 'Blue/Brown Brindle', 'Blue/Calico', 'Blue/Cream', 'Blue/Fawn', 'Blue/Gray', 'Blue/Orange', 'Blue/Silver', 'Blue/Tan', 'Blue/Tortie', 'Blue/Tricolor', 'Blue/White', 'Blue/Yellow Brindle'], 'Blue', inplace =True)
adoptions5.replace(['Brown', 'Brown Brindle', 'Brown Brindle/Black', 'Brown Brindle/Black Brindle', 'Brown Brindle/Blue', 'Brown Brindle/Blue Cream', 'Brown Brindle/Blue Tick', 'Brown Brindle/Brown', 'Brown Brindle/Brown Brindle', 'Brown Brindle/Brown Merle', 'Brown Brindle/Fawn', 'Brown Brindle/Gray', 'Brown Brindle/Liver Tick', 'Brown Brindle/Red Tick', 'Brown Brindle/Tan', 'Brown Brindle/Tricolor', 'Brown Brindle/White', 'Brown Merle', 'Brown Merle/Black', 'Brown Merle/Blue Merle', 'Brown Merle/Brown', 'Brown Merle/Brown Tabby', 'Brown Merle/Chocolate', 'Brown Merle/Gray', 'Brown Merle/Tan', 'Brown Merle/White', 'Brown Tabby', 'Brown Tabby/Agouti', 'Brown Tabby/Black', 'Brown Tabby/Black Brindle', 'Brown Tabby/Black Tabby', 'Brown Tabby/Blue', 'Brown Tabby/Brown', 'Brown Tabby/Brown Tabby', 'Brown Tabby/Calico', 'Brown Tabby/Cream', 'Brown Tabby/Gray', 'Brown Tabby/Gray Tabby', 'Brown Tabby/Lynx Point', 'Brown Tabby/Orange', 'Brown Tabby/Orange Tabby', 'Brown Tabby/Red', 'Brown Tabby/Silver', 'Brown Tabby/Tortie', 'Brown Tabby/White', 'Brown Tiger', 'Brown Tiger/Brown', 'Brown Tiger/White', 'Brown/Apricot', 'Brown/Black', 'Brown/Black Brindle', 'Brown/Black Smoke', 'Brown/Black Tabby', 'Brown/Blue', 'Brown/Blue Merle', 'Brown/Brown', 'Brown/Brown Brindle', 'Brown/Brown Merle', 'Brown/Brown Tabby', 'Brown/Buff', 'Brown/Chocolate', 'Brown/Cream', 'Brown/Fawn', 'Brown/Gold', 'Brown/Gray', 'Brown/Liver', 'Brown/Orange', 'Brown/Red', 'Brown/Red Merle', 'Brown/Red Tick', 'Brown/Seal Point', 'Brown/Silver', 'Brown/Tan', 'Brown/Tricolor', 'Brown/White', 'Brown/Yellow'], 'Brown', inplace =True)
adoptions5.replace(['Buff', 'Buff/Black', 'Buff/Brown', 'Buff/Cream', 'Buff/Gray', 'Buff/Red', 'Buff/Tan', 'Buff/White', 'Buff/Yellow'], 'Buff', inplace =True)
adoptions5.replace(['Calico', 'Calico Point', 'Calico Point/Gray', 'Calico Point/Lynx Point', 'Calico Point/White', 'Calico/Black', 'Calico/Blue Cream', 'Calico/Blue Tabby', 'Calico/Brown', 'Calico/Brown Tabby', 'Calico/Calico', 'Calico/Gray', 'Calico/Gray Tabby', 'Calico/Orange', 'Calico/Orange Tabby', 'Calico/Tricolor', 'Calico/White'], 'Calico', inplace =True)
adoptions5.replace(['Chocolate', 'Chocolate Point', 'Chocolate Point/Cream', 'Chocolate Point/White', 'Chocolate/Black', 'Chocolate/Black Smoke', 'Chocolate/Blue Tick', 'Chocolate/Brown', 'Chocolate/Brown Brindle', 'Chocolate/Brown Merle', 'Chocolate/Brown Tabby', 'Chocolate/Chocolate', 'Chocolate/Cream', 'Chocolate/Fawn', 'Chocolate/Gold', 'Chocolate/Gray', 'Chocolate/Liver Tick', 'Chocolate/Red', 'Chocolate/Red Tick', 'Chocolate/Seal Point', 'Chocolate/Tan', 'Chocolate/Tricolor', 'Chocolate/White', 'Chocolate/Yellow'], 'Chocolate', inplace =True)
adoptions5.replace(['Cream', 'Cream Tabby', 'Cream Tabby/Cream Tabby', 'Cream Tabby/Orange', 'Cream Tabby/Orange Tabby', 'Cream Tabby/White', 'Cream Tiger', 'Cream Tiger/Brown Tabby', 'Cream/Black', 'Cream/Blue', 'Cream/Blue Point', 'Cream/Brown', 'Cream/Brown Merle', 'Cream/Cream', 'Cream/Gold', 'Cream/Gray', 'Cream/Orange', 'Cream/Red', 'Cream/Red Tick', 'Cream/Seal Point', 'Cream/Silver', 'Cream/Tan', 'Cream/Tricolor', 'Cream/White', 'Cream/Yellow'], 'Cream', inplace =True)
adoptions5.replace(['Fawn', 'Fawn/Black', 'Fawn/Blue', 'Fawn/Brown', 'Fawn/Brown Brindle', 'Fawn/Chocolate', 'Fawn/Cream', 'Fawn/Gray', 'Fawn/Tan', 'Fawn/Tricolor', 'Fawn/White'], 'Fawn', inplace =True)
adoptions5.replace(['Flame Point', 'Flame Point/Cream', 'Flame Point/White'], 'Flame Point', inplace =True)
adoptions5.replace(['Gold', 'Gold/Black', 'Gold/Brown', 'Gold/Buff', 'Gold/Cream', 'Gold/Gold', 'Gold/Gray', 'Gold/Silver', 'Gold/Tan', 'Gold/White', 'Gold/Yellow'], 'Gold', inplace =True)
adoptions5.replace(['Gray', 'Gray Tabby', 'Gray Tabby/Black', 'Gray Tabby/Brown', 'Gray Tabby/Brown Tabby', 'Gray Tabby/Gray', 'Gray Tabby/Orange', 'Gray Tabby/White', 'Gray/Black', 'Gray/Blue', 'Gray/Blue Merle', 'Gray/Brown', 'Gray/Brown Brindle', 'Gray/Buff', 'Gray/Cream', 'Gray/Fawn', 'Gray/Gold', 'Gray/Gray', 'Gray/Gray Tabby', 'Gray/Orange', 'Gray/Red', 'Gray/Silver', 'Gray/Tan', 'Gray/Tortie', 'Gray/Tricolor', 'Gray/White'], 'Gray', inplace =True)
adoptions5.replace(['Lilac Point', 'Lilac Point/Black', 'Lilac Point/Cream', 'Lilac Point/Gray', 'Lilac Point/White'], 'Lilac Point', inplace =True)
adoptions5.replace(['Liver', 'Liver Tick', 'Liver Tick/Brown', 'Liver Tick/White', 'Liver/Blue', 'Liver/Brown Brindle', 'Liver/Buff', 'Liver/Chocolate', 'Liver/Liver Tick', 'Liver/Tan', 'Liver/White'], 'Liver', inplace =True)
adoptions5.replace(['Lynx Point', 'Lynx Point/Blue', 'Lynx Point/Brown Tabby', 'Lynx Point/Cream', 'Lynx Point/Gray', 'Lynx Point/Gray Tabby', 'Lynx Point/Tan', 'Lynx Point/Tortie Point', 'Lynx Point/White'], 'Lynx Point', inplace =True)
adoptions5.replace(['Orange', 'Orange Tabby', 'Orange Tabby/Apricot', 'Orange Tabby/Black', 'Orange Tabby/Brown', 'Orange Tabby/Cream', 'Orange Tabby/Orange', 'Orange Tabby/Orange Tabby', 'Orange Tabby/Tortie Point', 'Orange Tabby/White', 'Orange Tiger', 'Orange/Orange Tabby', 'Orange/Tan', 'Orange/White'], 'Orange', inplace =True)
adoptions5.replace(['Red', 'Red Merle', 'Red Merle/Black', 'Red Merle/Brown', 'Red Merle/Brown Merle', 'Red Merle/Red Merle', 'Red Merle/Tan', 'Red Merle/Tricolor', 'Red Merle/White', 'Red Tick', 'Red Tick/Black', 'Red Tick/Blue', 'Red Tick/Blue Tick', 'Red Tick/Brown', 'Red Tick/Brown Brindle', 'Red Tick/Brown Merle', 'Red Tick/Orange', 'Red Tick/Red', 'Red Tick/Tan', 'Red Tick/Tricolor', 'Red Tick/White', 'Red/Black', 'Red/Blue', 'Red/Brown', 'Red/Brown Brindle', 'Red/Buff', 'Red/Cream', 'Red/Gold', 'Red/Gray', 'Red/Red', 'Red/Red Merle', 'Red/Red Tick', 'Red/Silver', 'Red/Tan', 'Red/Tricolor', 'Red/White', 'Red/Yellow', 'Ruddy/Cream'], 'Red', inplace =True)
adoptions5.replace(['Sable', 'Sable/Black', 'Sable/Brown', 'Sable/Buff', 'Sable/Cream','Sable/Gray', 'Sable/Red', 'Sable/Red Merle', 'Sable/Tan', 'Sable/White'], 'Sable', inplace =True)
adoptions5.replace(['Seal Point', 'Seal Point/White', 'Seal Point/Brown', 'Seal Point/Cream', 'Seal Point/Buff', 'Seal Point/Gray'], 'Seal Point', inplace =True)
adoptions5.replace(['Silver', 'Silver Lynx Point', 'Silver Lynx Point/Gray', 'Silver Lynx Point/White', 'Silver Tabby', 'Silver Tabby/Black', 'Silver Tabby/White', 'Silver/Black', 'Silver/Blue', 'Silver/Brown', 'Silver/Chocolate', 'Silver/Cream', 'Silver/Gray', 'Silver/Red', 'Silver/Silver Tabby', 'Silver/Tan', 'Silver/White'], 'Silver', inplace =True)
adoptions5.replace(['Tan', 'Tan/Apricot', 'Tan/Black', 'Tan/Blue', 'Tan/Blue Merle', 'Tan/Blue Tick', 'Tan/Brown', 'Tan/Brown Brindle', 'Tan/Brown Merle', 'Tan/Buff', 'Tan/Chocolate Point', 'Tan/Cream', 'Tan/Cream Tabby', 'Tan/Fawn', 'Tan/Gold', 'Tan/Gray', 'Tan/Red', 'Tan/Red Merle', 'Tan/Red Tick', 'Tan/Silver', 'Tan/Tan', 'Tan/Tricolor', 'Tan/White', 'Tan/Yellow', 'Tan/Yellow Brindle'], 'Tan', inplace =True)
adoptions5.replace(['Torbie', 'Torbie/Black', 'Torbie/Blue Cream', 'Torbie/Blue Tabby', 'Torbie/Brown', 'Torbie/Brown Tabby', 'Torbie/Calico', 'Torbie/Gray', 'Torbie/Silver Tabby', 'Torbie/White'], 'Torbie', inplace =True)
adoptions5.replace(['Tortie', 'Tortie Point', 'Tortie Point/Blue', 'Tortie Point/Cream', 'Tortie Point/Lynx Point', 'Tortie Point/Seal Point', 'Tortie Point/White', 'Tortie/Black', 'Tortie/Black Smoke', 'Tortie/Blue', 'Tortie/Blue Cream', 'Tortie/Blue Tabby', 'Tortie/Brown', 'Tortie/Calico', 'Tortie/Gold', 'Tortie/Gray', 'Tortie/Orange', 'Tortie/Tortie', 'Tortie/White'], 'Tortie', inplace =True)
adoptions5.replace(['Tricolor', 'Tricolor/Black', 'Tricolor/Blue', 'Tricolor/Blue Merle', 'Tricolor/Blue Tick', 'Tricolor/Brown', 'Tricolor/Brown Brindle', 'Tricolor/Brown Merle', 'Tricolor/Calico', 'Tricolor/Chocolate', 'Tricolor/Cream', 'Tricolor/Fawn', 'Tricolor/Gray', 'Tricolor/Orange', 'Tricolor/Red', 'Tricolor/Red Tick', 'Tricolor/Silver', 'Tricolor/Tan', 'Tricolor/Tricolor', 'Tricolor/White',], 'Tricolor', inplace =True)
adoptions5.replace(['White', 'White/Agouti', 'White/Apricot', 'White/Black', 'White/Black Brindle', 'White/Black Smoke', 'White/Black Tabby', 'White/Black Tiger', 'White/Blue', 'White/Blue Cream', 'White/Blue Merle', 'White/Blue Tabby', 'White/Blue Tick', 'White/Blue Tiger', 'White/Brown', 'White/Brown Brindle', 'White/Brown Merle', 'White/Brown Tabby', 'White/Brown Tiger', 'White/Buff', 'White/Calico', 'White/Calico Point', 'White/Chocolate', 'White/Chocolate Point', 'White/Cream', 'White/Cream Tabby', 'White/Fawn', 'White/Flame Point', 'White/Gold', 'White/Gray', 'White/Gray Tabby', 'White/Gray Tiger', 'White/Lilac Point', 'White/Liver', 'White/Liver Tick', 'White/Lynx Point', 'White/Orange', 'White/Orange Tabby', 'White/Pink', 'White/Red', 'White/Red Merle', 'White/Red Tick', 'White/Seal Point', 'White/Silver', 'White/Silver Tabby', 'White/Tan', 'White/Tortie', 'White/Tortie Point', 'White/Tricolor', 'White/White', 'White/Yellow', 'White/Yellow Brindle'], 'White', inplace =True)
adoptions5.replace(['Yellow', 'Yellow Brindle', 'Yellow Brindle/Black', 'Yellow Brindle/Blue', 'Yellow Brindle/White', 'Yellow/Black', 'Yellow/Brown', 'Yellow/Cream', 'Yellow/Gray', 'Yellow/Orange', 'Yellow/Orange Tabby', 'Yellow/Tan', 'Yellow/White', 'Yellow/Yellow'], 'Yellow', inplace =True)

C:\Users\bueno\anaconda3\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [61]:
adoptions5.Color.sort_values().unique()

array(['Agouti', 'Apricot', 'Black', 'Blue', 'Brown', 'Buff', 'Calico',
       'Chocolate', 'Cream', 'Fawn', 'Flame Point', 'Gold', 'Gray',
       'Lilac Point', 'Liver', 'Lynx Point', 'Orange', 'Red', 'Sable',
       'Seal Point', 'Silver', 'Tan', 'Torbie', 'Tortie', 'Tricolor',
       'White', 'Yellow'], dtype=object)

In [62]:
adoptions5.head()

,Animal Type,Color,Year,Month,Sex,Age,AdoptionYN,typeR,sexR,ageR
0,Cat,Brown,19,May,Male,2-5 years,0,0,0,1.0
1,Dog,White,18,Jul,Male,0-1 years,1,1,0,0.0
3,Dog,Buff,16,Feb,Male,0-1 years,1,1,0,0.0
4,Cat,Orange,14,Mar,Male,0-1 years,0,0,0,0.0
5,Dog,Brown,20,Oct,Female,6-9 years,1,1,1,2.0


### Recode Color to a numeric variable

In [63]:
def color (series): 
    if series == "Agouti" : 
        return 0
    if series == "Apricot": 
        return 1
    if series == "Black" : 
        return 2
    if series == "Blue" : 
        return 3
    if series =="Brown":
        return 4
    if series =="Buff":
        return 5
    if series =="Calico":
        return 6
    if series =="Chocolate":
        return 7
    if series =="Cream":
        return 8
    if series =="Fawn":
        return 9
    if series =="Flame Point":
        return 10
    if series =="Gold":
        return 11
    if series =="Gray":
        return 12
    if series =="Lilac Point":
        return 13
    if series =="Liver":
        return 14
    if series =="Lynx Point":
        return 15
    if series =="Orange":
        return 16
    if series =="Red":
        return 17
    if series =="Sable":
        return 18
    if series =="Seal Point":
        return 19
    if series =="Silver":
        return 20
    if series =="Tan":
        return 21
    if series =="Torbie":
        return 22
    if series =="Tortie":
        return 23
    if series =="Tricolor":
        return 24
    if series =="White":
        return 25
    if series =="Yellow":
        return 26
         
adoptions5["colorR"] = adoptions5["Color"].apply(color)

<ipython-input-63-720d49238649>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adoptions5["colorR"] = adoptions5["Color"].apply(color)


In [64]:
adoptions5.head()

,Animal Type,Color,Year,Month,Sex,Age,AdoptionYN,typeR,sexR,ageR,colorR
0,Cat,Brown,19,May,Male,2-5 years,0,0,0,1.0,4
1,Dog,White,18,Jul,Male,0-1 years,1,1,0,0.0,25
3,Dog,Buff,16,Feb,Male,0-1 years,1,1,0,0.0,5
4,Cat,Orange,14,Mar,Male,0-1 years,0,0,0,0.0,16
5,Dog,Brown,20,Oct,Female,6-9 years,1,1,1,2.0,4


### Convert adoptions5 to csv for exploratory analysis in Tableau

In [65]:
os.makedirs('data', exist_ok=True)  # create "data" folder in Final Projects folder
adoptions5.to_csv('data/adoptions5.csv')   # convert adoptions5 from a pandas dataframe to a csv file and save in "data" folder

## Analyses

[Analyses in R](http://localhost:8888/notebooks/OneDrive/Documents/GitHub/FinalProject/FinalProject-Analyses.ipynb)

[Analysis in Python](http://localhost:8888/notebooks/OneDrive/Documents/GitHub/FinalProject/FinalProject-AnalysesInPython.ipynb)